In [20]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [63]:
!pip install fuzzywuzzy

In [21]:
url_m="https://raw.githubusercontent.com/Praful2000/YoutubeLectures/master/Movie%20KNN/movies.csv"
movies=pd.read_csv(url_m)
url_r='https://raw.githubusercontent.com/Praful2000/YoutubeLectures/master/Movie%20KNN/ratings.csv'
ratings=pd.read_csv(url_r)

In [22]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [23]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [25]:
movies=movies.merge(ratings,on="movieId")
movies.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5


In [27]:
# ratings=ratings.drop(["timestamp"],axis=1)
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


 NOW I WANT A MATRIX OF ROW AS MOVIEID AS ROW COLUMN AND COLUMN HEADING AS USERID AND RATING AS VALUE

In [38]:
# to check how many users are there 
ratings.userId.nunique()

610

In [43]:
movie_users=ratings.pivot(index="movieId",columns="userId",values="rating").fillna(0)
movie_users.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
from scipy.sparse import csr_matrix
# sparse matrix is a matrix that no of 0 entries greater than the no of non zeros entries

In [51]:
mat_movies=csr_matrix(movie_users.values)
mat_movies

<9724x610 sparse matrix of type '<class 'numpy.float64'>'
	with 100836 stored elements in Compressed Sparse Row format>

In [60]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(metric="cosine",algorithm="brute",n_neighbors=20)
# cosine matric to found similarity between each matrix and brute force algorithm for each cell similarity
model.fit(mat_movies)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [65]:
from fuzzywuzzy import process
# Ai libraray to increase our searching process

In [82]:
def recommender(movie_name,data,n):
    idx=process.extractOne(movie_name,movies['title'])[2]
    print('Movie Selected: ',movies['title'][idx],"Index: ",idx)
    print("Searching for recommendations...")
    distances,indices=model.kneighbors(data[idx],n_neighbors=n)
    # here distance is cosine distance between different vectors
    for i  in indices:
        print(movies["title"][i]. where(i!=idx))

In [83]:
recommender('toy story',mat_movies,10)
# mactched with ,how much similarity and index

Movie Selected:  Toy Story (1995) Index:  0
Searching for recommendations...
0                                      NaN
2353                     Pocahontas (1995)
418     Father of the Bride Part II (1995)
615                       GoldenEye (1995)
224                         Jumanji (1995)
314                         Jumanji (1995)
322                         Jumanji (1995)
910                          Casino (1995)
546                         Sabrina (1995)
963           Sense and Sensibility (1995)
Name: title, dtype: object
